## *De novo* TSS finding by CROWN-Seq and ReCappable-Seq

Jianheng Liu (Fox) @ Jaffrey Lab, May 31st, 2023

Concat: jil4026@med.cornell.edu

**Usage:** Modify `Prepartion` section to fit your computer. Modify `Variable` section before each run.All outputs will be saved in the `workpath`. Logs will be saved in the notebook. Don't close the `Notebook` before it finished (keep the backend at least)!

**Note 1:** In Jupyter notebook, `!` means using `bash` in the cell, and `$` means using python variables.

**Note 2:** This panel notebook should be compatible with `Python 3`. But please use `Python 2` to run the analytic scripts.

**Note 3:** I used Gencode v34 in the article for compatiblity. Here I use Ensembl release 104 for illustration. Newer annotations are normally better.

**Note 4:** This notebook is for paired-end reads. If you are using single-end, switch to the scripts end with `_SE.py`. And don't forget to change the parameters.

In [1]:
# local time
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%D %H:%M:%S")
print("Started:", current_time)

# Show notebook directory
!pwd -P

Started: 05/31/23 15:34:57
/home/fox/Projects/CROWN-Seq_example


### 0. Variable here

In [2]:
name = "HEK_WT" # file name prefix
folder = "WT_run/"
path = "./" # output path: workpath=path/folder
workpath = path + "/" + folder + "/"
read1 = "HEK293T_rep1_example.R1.fastq.gz" # fastq or fastq.gz
read2 = "HEK293T_rep1_example.R2.fastq.gz"

### 1. Preparation

In [3]:
ref_genome_index = "/home/fox/Database/GLORI_index/GRCh38_r104/hisat2_index/"
ref_genome = "/home/fox/Database/GLORI_index/GRCh38_r104/hisat2_index/Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa"
tx_db_exons = "/home/fox/Database/Genome/Ensembl/GRCh38_r104/Homo_sapiens.GRCh38.104.noheader.tx.bed"
tx_db_tss = "/home/fox/Database/Genome/Ensembl/GRCh38_r104/Homo_sapiens.GRCh38.104.noheader.anno.tss"
faidx = "/home/fox/Database/GLORI_index/GRCh38_r104/hisat2_index/Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa.fai"
annot = "/home/fox/Database/Genome/Ensembl/GRCh38_r104/Homo_sapiens.GRCh38.104.noheader.anno"

**Software** (For python, use the fixed version)

In [4]:
python = '/home/fox/Software/bin/python'
bowtie2 = '/home/fox/Software/bin/python2'
samtools = '/home/fox/Software/samtools/1.16/bin/samtools'
cutadapt = '/home/fox/Software/bin/cutadapt'
hisat2_path = '/home/fox/Software/hisat2/2.1.0/'
umitools = '/home/fox/Software/bin/umi_tools'
bedtools = '/home/fox/Software/bedtools/2.29.1/bedtools'

**Scripts**

In [5]:
hisat2_script = "../A2G_hisat2.py"
bam2bed_script  = "../Align_to_cap_starts_PE.py"
annot_script_1 = "../collaspe_bed_annotations_v2.py"
annot_script_2 = "../collaspe_bed_annotations_fix_other_exons.py"
bed_annotate_base = "../genome_flanking_v3_bed_annot.py"
bed_annotate_tpm = "../get_tpm.py"
tss_filters = "../ReCappabble-seq_caller_ben_version_v2.py"

**Create the workpath if not exist, then move to the workpath**

In [6]:
import os
if os.path.isdir(workpath) == False:
    os.mkdir(workpath)
os.chdir(workpath)

### 2. QC

In [7]:
adapter_1 = "AGATCGGAAGAGCACACGTC"
adapter_2 = "ATATNNNNNNNNAGATCGGAAGAGCGTCGTG"  # UMI = NNNNNNNNATAT

cutadapt_out_r1 = "read1.cutadapt.fastq"
cutadapt_out_r2 = "read2.cutadapt.fastq"

!$cutadapt -m 32 -j 4 -q 20 -e 0.25 -a $adapter_1 -A $adapter_2 -o $cutadapt_out_r1 -p $cutadapt_out_r2 ../$read1 ../$read2

This is cutadapt 4.1 with Python 3.9.14
Command line parameters: -m 32 -j 4 -q 20 -e 0.25 -a AGATCGGAAGAGCACACGTC -A ATATNNNNNNNNAGATCGGAAGAGCGTCGTG -o read1.cutadapt.fastq -p read2.cutadapt.fastq ../HEK293T_rep1_example.R1.fastq.gz ../HEK293T_rep1_example.R2.fastq.gz
Processing paired-end reads on 4 cores ...
Done           00:00:07     1,000,000 reads @   7.9 µs/read;   7.61 M reads/minute
Finished in 7.89 s (8 µs/read; 7.61 M reads/minute).

=== Summary ===

Total read pairs processed:          1,000,000
  Read 1 with adapter:                 971,361 (97.1%)
  Read 2 with adapter:                 327,934 (32.8%)

== Read fate breakdown ==
Pairs that were too short:             799,979 (80.0%)
Pairs written (passing filters):       200,021 (20.0%)

Total basepairs processed:   202,000,000 bp
  Read 1:   101,000,000 bp
  Read 2:   101,000,000 bp
Quality-trimmed:                 974,581 bp (0.5%)
  Read 1:       468,185 bp
  Read 2:       506,396 bp
Total written (filtered):     26,521

In [8]:
# Extract UMI
UMI_out_r1 = "read1.UMI.fastq"
UMI_out_r2 = "read2.UMI.fastq"

# A problem about UMI-tools...

umitools_cmd = "{umitools} extract -p NNNNNNNNNNNN -I {cutadapt_out_r1} -S {UMI_out_r1} --read2-in {cutadapt_out_r2} --read2-out {UMI_out_r2}".format(umitools=umitools, cutadapt_out_r1=cutadapt_out_r1, cutadapt_out_r2=cutadapt_out_r2, UMI_out_r1=UMI_out_r1, UMI_out_r2=UMI_out_r2)

import os
os.system(umitools_cmd)


# UMI-tools version: 1.1.1
# output generated by extract -p NNNNNNNNNNNN -I read1.cutadapt.fastq -S read1.UMI.fastq --read2-in read2.cutadapt.fastq --read2-out read2.UMI.fastq
# job started at Wed May 31 15:35:06 2023 on orion -- 99e4f7ca-9ac5-4d87-bb13-84ba93b04a62
# pid: 1659347, system: Linux 5.15.0-69-generic #76~20.04.1-Ubuntu SMP Mon Mar 20 15:54:19 UTC 2023 x86_64
# blacklist                               : None
# compresslevel                           : 6
# correct_umi_threshold                   : 0
# either_read                             : False
# either_read_resolve                     : discard
# error_correct_cell                      : False
# extract_method                          : string
# filter_cell_barcode                     : None
# filter_cell_barcodes                    : False
# filter_umi                              : None
# filtered_out                            : None
# filtered_out2                           : None
# ignore_suffix                     

0

### 2. Mapping

In [9]:
!$python $hisat2_script -F $UMI_out_r1 -R $UMI_out_r2 -o hisat2_genome -I $ref_genome_index --index-prefix HISAT2 --hisat2-path $hisat2_path --del-convert --del-sam

[2023-05-31 15:35:12]Converting read1.UMI.fastq, A2G...
[2023-05-31 15:35:12]Converting read2.UMI.fastq, T2C...
[2023-05-31 15:35:13]Mapping with hisat2, TEMP prefix: hisat2_1659426
[2023-05-31 15:35:28]T2C report:
[2023-05-31 15:35:28]A2G report:
[2023-05-31 15:35:28]Handling SAM outputs...
[2023-05-31 15:35:45]Completed successfully:
 Total reads: 200021
 Unique mapping: 98136 (49.063%)
   A2G: 50229 (25.11%)
   T2C: 47907 (23.95%)
 Multiple mapping: 29860 (14.928%)
 Unmapped: 72025 (36.009%)


In [10]:
!$samtools sort -@ 4 -m 4G -o hisat2_genome.sorted.bam hisat2_genome.bam
!$samtools index hisat2_genome.sorted.bam

[bam_sort_core] merging from 0 files and 4 in-memory blocks...


In [11]:
!$umitools dedup --stdin=hisat2_genome.sorted.bam --log=umi.logs --method=unique > hisat2_genome.sorted.umi.bam

In [12]:
!samtools sort -@ 4 -m 4G -o hisat2_genome.sorted.umi.sorted.bam  hisat2_genome.sorted.umi.bam
!samtools index hisat2_genome.sorted.umi.sorted.bam 

[bam_sort_core] merging from 0 files and 4 in-memory blocks...


### 3. Fetch 5' ends of the reads

In [13]:
bed_output = "{}.bed".format(name)
sorted_bed = "{}.sorted.bed".format(name)

!$python $bam2bed_script hisat2_genome.sorted.umi.sorted.bam > $bed_output
!$bedtools sort -i $bed_output > $sorted_bed

### 4. Find the closest annotations for the 5' ends

In [14]:
bed_tss_annot = "{}.closest.tss.bed".format(name)
bed_tx_annot = "{}.closest.tx.bed".format(name)

!$bedtools closest -s -D b -a $sorted_bed -b $tx_db_tss > $bed_tss_annot
!$bedtools closest -s -D b -a $sorted_bed -b $tx_db_exons > $bed_tx_annot

### 5. Find the best annotation for the 5' ends

First, annotate by the closest TSS, ambiguous annoations are in the format "*{name of the gene}_{distance}"

In [15]:
bed_best_tss = "{}.byTSS.bed".format(name)

!$python $annot_script_1 -i $bed_tss_annot -a $annot  -o $bed_best_tss

Second, the ambiguous annotations will be re-annotated by the transcripts. If the site is still ambiguous, the will be annotated by "*{name of the gene}_{distance}" again.

In [16]:
bed_best_tss_tx = "{}.byTSS.byTx.bed".format(name)

!$python $annot_script_2 -i $bed_best_tss -I $bed_tx_annot -a $annot -o $bed_best_tss_tx

### 6. Simple annotations for the base and TPM

In [17]:
bed_with_base = "{}.byTSS.byTx.base.bed".format(name)
bed_with_tpm = "{}.byTSS.byTx.base.tpm.bed".format(name)

!$python $bed_annotate_base -f $ref_genome -F 0 -i $bed_best_tss_tx > $bed_with_base
!$python $bed_annotate_tpm $bed_with_base > $bed_with_tpm

### 7. Last step, find out which one is high-confident

In [18]:
output_prefix = "{}.called".format(name)

!$python $tss_filters -i $bed_with_tpm -o $output_prefix

  0%|                                          | 3/6495 [00:00<09:35, 11.27it/s]/home/fox/Projects/CROWN-Seq_example/WT_run/../ReCappabble-seq_caller_ben_version_v2.py:71: RuntimeWarning: invalid value encountered in divide
  zscores = (tpms - np.mean(tpms))/np.std(tpms)
100%|█████████████████████████████████████| 6495/6495 [00:01<00:00, 6412.94it/s]


Finally, there will be two outputs:

The high-confident sites will be stored in:

`output_prefix`.ben.passed.bed 

While the sites do not pass the filters will be

`output_prefix`.ben.filtered_out.bed

Both of these two files are important for further analysis.

### 8. When and where am I

In [19]:
!pwd -P
!ls -l

/home/fox/Projects/CROWN-Seq_example/WT_run
total 251156
-rw-rw-r-- 1 fox fox   765425 May 31 15:36 HEK_WT.bed
-rw-rw-r-- 1 fox fox  1695512 May 31 15:36 HEK_WT.byTSS.bed
-rw-rw-r-- 1 fox fox  1603519 May 31 15:37 HEK_WT.byTSS.byTx.base.bed
-rw-rw-r-- 1 fox fox  2138272 May 31 15:37 HEK_WT.byTSS.byTx.base.tpm.bed
-rw-rw-r-- 1 fox fox  1546621 May 31 15:37 HEK_WT.byTSS.byTx.bed
-rw-rw-r-- 1 fox fox  2751484 May 31 15:37 HEK_WT.called.ben.filtered_out.bed
-rw-rw-r-- 1 fox fox   753307 May 31 15:37 HEK_WT.called.ben.passed.bed
-rw-rw-r-- 1 fox fox  2598124 May 31 15:36 HEK_WT.closest.tss.bed
-rw-rw-r-- 1 fox fox  9730729 May 31 15:36 HEK_WT.closest.tx.bed
-rw-rw-r-- 1 fox fox   765425 May 31 15:36 HEK_WT.sorted.bed
-rw-rw-r-- 1 fox fox  8740663 May 31 15:35 hisat2_genome.bam
-rw-rw-r-- 1 fox fox  7445418 May 31 15:35 hisat2_genome.multimappers.bam
-rw-rw-r-- 1 fox fox  6657445 May 31 15:35 hisat2_genome.sorted.bam
-rw-rw-r-- 1 fox fox  1687456 May 31 15:35 hisat2_genome.sorted.bam.bai
-rw

In [20]:
now = datetime.now()
current_time = now.strftime("%D %H:%M:%S")
print("Finished:", current_time)

Finished: 05/31/23 15:37:17


## Don't forget to save the Notebook.